---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1079]:
import pandas as pd
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [1134]:
def date_sorter():

    df2 = pd.DataFrame(doc, columns = ['txt'])
    
    # mm/dd/yy lub mm/dd/yyyy
    a = df.str.extract(r"(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})")
    a.reset_index(inplace=True)
    a = a.rename(columns = {0 : "month1", 1 : "day1", 2 : "year1"})
    a['year1']=a['year1'].apply(lambda x: '19'+x if len(str(x))<=2 else x)
    a = a.drop(['index'], axis = 1)
    
    # mm/yy lub mm/yyyy
    b = df.str.extract(r'(\d{1,2})[/](\d{4})')
    b.reset_index(inplace=True)
    b = b.rename(columns = {0 : "month2", 1 : "year2"})
    b = b.drop(['index'], axis = 1)

    # yyyy
    # add "r'^(\d{4})[^0-9]'"
    c = pd.DataFrame(df.str.extract(r'[a-z]?[^0-9](\d{4})[^0-9]'), columns = ['year3'])
    c.reset_index(inplace=True)
    c = c.drop(['index'], axis = 1)
        
    # ... Jan... slowno liryczny opis miesiaca
    d = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    d.reset_index(inplace=True)
    d.set_index(['level_0'], inplace = True)
    d = d.rename(columns = {0 : "day4", 1 : "month4", 2 : "year4"})
    d = d.drop(['match'], axis = 1)

    # Jan... slowno liryczny opis miesiaca
    e = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.,]* )((?:\d{1,2}[?:, -]*)\d{4})')
    e.reset_index(inplace=True)
    e.set_index(['level_0'], inplace = True)
    e[1] = e[1].str.replace(' ', ',')
    e[1] = e[1].str.split(',')
    e['day'] = e[1]
    e['year5'] = e['day'].str[-1]
    e['day5'] = e['day'].str[0]
    e = e.rename(columns = {0 : "month5"})
    e = e.drop(['match', 1, 'day'], axis = 1)
    
    f = pd.DataFrame(df.str.extract(r'^(\d{4})[^0-9]'), columns = ['year6'])
    f.reset_index(inplace=True)
    f = f.drop(['index'], axis = 1)
    
    fin = df2 \
            .merge(a, left_index = True, right_index = True, how = 'left') \
                .merge(b, left_index = True, right_index = True, how = 'left') \
                    .merge(c, left_index = True, right_index = True, how = 'left') \
                        .merge(d, left_index = True, right_index = True, how = 'left') \
                            .merge(e, left_index = True, right_index = True, how = 'left') \
                                .merge(f, left_index = True, right_index = True, how = 'left') \
    
    fin['day4'] = np.where(~pd.isnull(fin['day4']), fin['day4'], '01')
    
    fin['dt1'] = fin['year1'] + "-" + fin['month1'] + "-" + fin['day1']
    
    fin['dt2'] = fin['year2'] + "-" + fin['month2'] + "-01"
    
    fin['dt3'] = fin['year3'].str.rstrip() + "-01-01"
    
    fin['dt4'] = (fin['year4'] + "-" + fin['month4'].str.rstrip() + "-" + fin['day4']).str.rstrip()
    
    fin['dt5'] = (fin['year5'] + "-" + fin['month5'].str.rstrip() + "-" + fin['day5']).str.rstrip()
    
    fin['dt6'] = fin['year6'].str.rstrip() + "-01-01"
    
    fin['dt_fin'] = 0
    
    fin['dt_fin'] = np.where(~pd.isnull(fin['dt1']), fin['dt1'], 
                                     np.where(~pd.isnull(fin['dt4']), fin['dt4'],
                                              np.where(~pd.isnull(fin['dt5']), fin['dt5'],
                                                       np.where(~pd.isnull(fin['dt2']), fin['dt2'], 
                                                                np.where(~pd.isnull(fin['dt3']), fin['dt3'],
                                                                         np.where(~pd.isnull(fin['dt6']), fin['dt6'],
                                                                         '1990-01-01'
                                    ))))))
    
    fin['dt_fin'] = fin['dt_fin'].str.replace('.', '')
    fin['dt_fin'] = fin['dt_fin'].str.replace(',', '')

    dates = pd.to_datetime(fin['dt_fin'], errors='coerce')
    
    dt = []
    for i in dates:
        if i is pd.NaT:
            dt.append(pd.to_datetime('1990-01-01'))
        else:
            dt.append(i)

    sort = pd.Series([i[0] for i in sorted(enumerate(dt), key=lambda x:x[1])],np.arange(500))

    return sort

In [1135]:
date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:43: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFram

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64